# Keras Intensive Run

Instructions
1. Define the global variables;
2. Run the second snippet. It builds the network and saves the output to a folder displayed on the console.

In [83]:
# TEST VARIABLES
labelsGroup = [
    ["happy", "sad"],
]
iterationsGroup = [100]
hiddenLayersGroup = [
    [144],
]
batchSize = 128
hyperParametersGroup = [0, 0.01]

In [91]:
# GLOBAL VARIABLES
labelsGroup = [
    ["angry", "fear", "happy", "neutral", "sad", "surprise"],
    # ["fear", "angry", "surprise", "neutral"],
    # ["fear", "angry", "surprise"],
    # ["fear", "angry"],
    # ["fear", "sad", "happy"],
    # ["happy", "angry", "neutral"],
    # ["happy", "sad"]
]
iterationsGroup = [5000]
hiddenLayersGroup = [
    # [144],
    [144, 144],
    [300,200,100,50],
    [300,100,100,100],
]
batchSize = 128
hyperParametersGroup = [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]

In [92]:
def confusionMatrix(labels, pred, y):
    
    confusionTFPN = {}
    confusionLabels = {}
    
    # Foreach emotion
    for ie in range(len(labels)):
        # TRUE AND FALSE POSITIVES AND NEGATIVES (TFPN)
        
        # Get indexes where emotion was predicted
        ieIndexesPred = [i for i in range(pred.size) if pred[i]==ie]
        ieIndexesNotPred = [i for i in range(pred.size) if pred[i]!=ie]

        # Predicted, and Actual
        TP = sum(pred[:,np.newaxis][ieIndexesPred]==y[ieIndexesPred])
        # Predicted, but not actual
        FP = sum(pred[:,np.newaxis][ieIndexesPred]!=y[ieIndexesPred])
        # Not predicted, but actual
        FN = sum(y[ieIndexesNotPred]==ie)
        # Not predicted and not actual
        TN = sum(y[ieIndexesNotPred]!=ie)

        TP = TP[0] if TP else 0
        FP = FP[0] if FP else 0
        FN = FN[0] if FN else 0
        TN = TN[0] if TN else 0
        
        confusionTFPN[labels[ie]] = {
            'TP': TP,
            'FP': FP,
            'FN': FN,
            'TN': TN
        }
        
        # CONFUSION WITH OTHER labels (Confusion)
        # For emotion e
        # Count the number of predictions made for all classes
        # Foreach emotion, check how many times it has been predicted 
        
        # Get indexes where emotion is real
        ieIndexesY = [i for i in range(y.size) if y[i]==ie]
        
        # Foreach matching prediction, check what emotion was predicted
        confusionLabels[labels[ie]] = {
            e: sum(pred[:,np.newaxis][ieIndexesY]==labels.index(e))[0] for e in labels
        }
        
    return confusionTFPN, confusionLabels

def outputConfusionTables(labels, testExamplesNumber, confusionTFPN, confusionLabels, axs, axsLine, axsTitle):
     # TABLES
    rows = tuple(labels)
    
    # TABLES / TFPN
    # Output confusion matrix as plot table
    data = [
        [
            f"{vals['TP']} ({vals['TP']/testExamplesNumber*100:.2f}%)",
            f"{vals['TN']} ({vals['TN']/testExamplesNumber*100:.2f}%)",
            f"{vals['TP']+vals['TN']} ({(vals['TP']+vals['TN'])/testExamplesNumber*100:.2f}%)",
            f"{vals['FP']} ({vals['FP']/testExamplesNumber*100:.2f}%)",
            f"{vals['FN']} ({vals['FN']/testExamplesNumber*100:.2f}%)",
            f"{vals['FP']+vals['FN']} ({(vals['FP']+vals['FN'])/testExamplesNumber*100:.2f}%)",
            vals['FP']+vals['FN']+vals['TP']+vals['TN']
        ]
        for _, vals in confusionTFPN.items()
    ]
    columns = ['TP (%)', 'TN (%)', 'T (%)', 'FP (%)', 'FN (%)', 'F (%)', f"Total"]
    
    # Colorize cells depending on value
    cellColours=plt.cm.Blues([
        [
            vals['TP']/testExamplesNumber,
            vals['TN']/testExamplesNumber,
            0,
            vals['FP']/testExamplesNumber,
            vals['FN']/testExamplesNumber,
            0,
            0
        ]
        for _, vals in confusionTFPN.items()
    ])
        
    the_table = axs[axsLine].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    # the_table.scale(1.2, 1)
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(20)
    axs[axsLine].axis('off')
    axs[axsLine].axis('tight')
    axs[axsLine].set_title(f"Confusion matrix {axsTitle}", fontsize=20, pad=0)    
    
    # TABLES / Confusion    
    # Output confusion matrix as plot table
    data = [[o for _,o in others.items()] for _,others in confusionLabels.items()]
    columns = rows
    
    # Colorize cells depending on value
    vals = [o for _,others in confusionLabels.items() for _,o in others.items()]
    normal = plt.Normalize(min(vals)-1, max(vals)+1)
    cellColours=plt.cm.Blues(normal(data))
        
    the_table = axs[axsLine+1].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    
    the_table.set_fontsize(20)
    # the_table.scale(1, 4)
    axs[axsLine+1].axis('off')
    axs[axsLine+1].axis('tight')
    axs[axsLine+1].set_title(f"True/Predicted {axsTitle}", fontsize=20)

In [ ]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#to load matlab mat files
from scipy.io import loadmat
# for saving metrics
import os 
import json
# for neural network
import tensorflow as tf
from tensorflow import keras

if not os.path.exists('./metrics'):
    os.makedirs('./metrics')

# FOR EACH LABEL GROUP
for labelNames in labelsGroup:
    # LOAD FILES
    print("\n\nLABELS", ', '.join(labelNames))

    # Load Training/Dev/Test data
    mat=loadmat(f"../datasets/train/{'_'.join(labelNames)}.mat")
    X, y = mat["X"], mat["y"]
    matDev=loadmat(f"../datasets/dev/{'_'.join(labelNames)}.mat")
    X_valid, y_valid = matDev["X"], matDev["y"]
    matTest=loadmat(f"../datasets/test/{'_'.join(labelNames)}.mat")
    X_test, y_test = matTest["X"], matTest["y"]
    
    # NUMBERS
    m = X.shape[0] # number of training examples
    labels = np.max(y)+1 # number of labels
    features = X.shape[1] # number of features per example

    # Output user feedback
    print(f"Loaded {m} traing examples with {labels} labels, each with {features} features (pixels).")
    print(f"Loaded {X_valid.shape[0]} dev examples.")
    print(f"Loaded {X_test.shape[0]} test examples.")
    
    print()
    
    # Create folder for metrics
    labelFolder = f"./metrics/{'_'.join(labelNames)}"
    if not os.path.exists(labelFolder):
        os.makedirs(labelFolder)
    if not os.path.exists(f"{labelFolder}/history"):
        os.makedirs(f"{labelFolder}/history")
    if not os.path.exists(f"{labelFolder}/predict"):
        os.makedirs(f"{labelFolder}/predict")
    if not os.path.exists(f"{labelFolder}/archive"):
        os.makedirs(f"{labelFolder}/archive")
    
    # FOREACH HIDDEN LAYER
    for hiddenLayers in hiddenLayersGroup:
        # FOR EACH HYPER PARAMETER
        for hyperParameter in hyperParametersGroup:
            # FOR EACH ITERATION NUMBER
            for iterations in iterationsGroup:
                
                hiddenLayersText = '_'.join(map(str, hiddenLayers)) if len(hiddenLayers)>1 else hiddenLayers[0]
                print(hiddenLayersText)
                combinationName = f"{iterations}iter_{batchSize}batchS_{hyperParameter}hyper_{len(hiddenLayers)}hlayers__{hiddenLayersText}"
                folder = f"{labelFolder}/archive/{combinationName}/"
                if not os.path.exists(folder):
                    os.makedirs(folder)

                print(f"\nBuilding network for {iterations} iterations and batch size of {batchSize} and {len(hiddenLayers)} hidden layers: {hiddenLayersText}...")
                print("With regularization!" if hyperParameter else "Without regularization!")
                print()

                # BUILD NETWORK
                # Create model
                modelSeq = []
                # Flattens each image (48x48) to 1x2304
                modelSeq.append(keras.layers.Flatten(input_shape = [48, 48]))
                # Hidden layers with relu activation function
                for h in hiddenLayers:                        
                    modelSeq.append(keras.layers.Dense(
                        h, 
                        activation = "relu", 
                        bias_regularizer= keras.regularizers.l2(hyperParameter) if hyperParameter else None
                    ))
                # Output layer with softmax activation function
                modelSeq.append(keras.layers.Dense(
                    labels, 
                    activation = "softmax",
                    bias_regularizer= keras.regularizers.l2(hyperParameter) if hyperParameter else None
                ))

                model = keras.models.Sequential(modelSeq)

                # Compile model
                model.compile(
                    # Using sparse categorical crossentropy loss function
                    loss = "sparse_categorical_crossentropy",
                    # Using stochastic gradient descent as gradient descent
                    optimizer = "sgd",
                    # In addition to cost, we want accuracy to help understanding how the model is working 
                    metrics = ["accuracy"]
                )

                # Train the network
                history = model.fit(
                    X,
                    y,
                    epochs = iterations,
                    batch_size = batchSize,
                    validation_data = (X_valid, y_valid)
                )

                # METRICS (SAVE TO FILE)

                # Model training history
                with open(f"{folder}/history.json", "w") as f:
                    json.dump(history.history, f)

                # Test model evaluation
                with open(f"{folder}/evaluation.json", "w") as f:
                    json.dump(model.evaluate(X_test, y_test, return_dict=True), f)

                # Prediction for test data
                with open(f"{folder}/predict.json", "w") as f:
                    json.dump(model.predict(X_test).tolist(), f)

                print(f"\nModel trained and metrics saved to {folder}!")

                # GENERATE GRAPHS AND SAVE TO FILE

                # Model training history
                pd.DataFrame(history.history).plot(figsize = (16, 10))
                plt.grid(True)
                plt.gca().set_ylim(0, 1)
                plt.title("Neural network training metrics")
                plt.savefig(f"{labelFolder}/history/{combinationName}.png")

                # Prediction for test data
                predict = model.predict(X_test)
                y_pred = np.array([np.argmax(p) for p in predict])
                confusionTFPN, confusionLabels = confusionMatrix(labelNames, y_pred, y_test)
                fig, axs = plt.subplots(2,1)
                outputConfusionTables(labelNames, y_test.size, confusionTFPN, confusionLabels, axs, 0, "WITHOUT regularization")
                fig.set_size_inches(18,6*len(labelNames))
                fig.subplots_adjust(left=0.2, top=20)
                fig.tight_layout()
                fig.savefig(f"{labelFolder}/predict/{combinationName}.png")



LABELS angry, fear, happy, neutral, sad, surprise
Loaded 6000 traing examples with 6 labels, each with 2304 features (pixels).
Loaded 1200 dev examples.
Loaded 1200 test examples.

144_144

Building network for 5000 iterations and batch size of 128 and 2 hidden layers: 144_144...
Without regularization!

Epoch 1/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.8012 - accuracy: 0.1906 - val_loss: 1.7744 - val_accuracy: 0.1892
Epoch 2/5000
47/47 [==============================] - 0s 4ms/step - loss: 1.7632 - accuracy: 0.2220 - val_loss: 1.7562 - val_accuracy: 0.2450
Epoch 3/5000
47/47 [==============================] - 0s 3ms/step - loss: 1.7432 - accuracy: 0.2565 - val_loss: 1.7493 - val_accuracy: 0.2525
Epoch 4/5000
47/47 [==============================] - 0s 3ms/step - loss: 1.7321 - accuracy: 0.2639 - val_loss: 1.7372 - val_accuracy: 0.2683
Epoch 5/5000
47/47 [==============================] - 0s 3ms/step - loss: 1.7193 - accuracy: 0.2733 - val_loss: 1.7329 - val

Epoch 49/5000
47/47 [==============================] - 0s 8ms/step - loss: 1.5066 - accuracy: 0.4046 - val_loss: 1.6194 - val_accuracy: 0.3375
Epoch 50/5000
47/47 [==============================] - 0s 7ms/step - loss: 1.5007 - accuracy: 0.4098 - val_loss: 1.6213 - val_accuracy: 0.3567
Epoch 51/5000
47/47 [==============================] - 0s 7ms/step - loss: 1.5248 - accuracy: 0.3979 - val_loss: 1.6241 - val_accuracy: 0.3492
Epoch 52/5000
47/47 [==============================] - 0s 7ms/step - loss: 1.5101 - accuracy: 0.4046 - val_loss: 1.6209 - val_accuracy: 0.3567
Epoch 53/5000
47/47 [==============================] - 0s 7ms/step - loss: 1.4969 - accuracy: 0.4221 - val_loss: 1.6209 - val_accuracy: 0.3333
Epoch 54/5000
47/47 [==============================] - 0s 7ms/step - loss: 1.5025 - accuracy: 0.4019 - val_loss: 1.6247 - val_accuracy: 0.3558
Epoch 55/5000
47/47 [==============================] - 0s 7ms/step - loss: 1.5113 - accuracy: 0.4010 - val_loss: 1.6395 - val_accuracy: 0.3217

47/47 [==============================] - 0s 6ms/step - loss: 1.3704 - accuracy: 0.4686 - val_loss: 1.6575 - val_accuracy: 0.3542
Epoch 107/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.3734 - accuracy: 0.4627 - val_loss: 1.6675 - val_accuracy: 0.3425
Epoch 108/5000
47/47 [==============================] - 0s 7ms/step - loss: 1.3601 - accuracy: 0.4945 - val_loss: 1.6662 - val_accuracy: 0.3525
Epoch 109/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.3705 - accuracy: 0.4776 - val_loss: 1.6035 - val_accuracy: 0.3600
Epoch 110/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.3560 - accuracy: 0.4884 - val_loss: 1.6425 - val_accuracy: 0.3517
Epoch 111/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.3614 - accuracy: 0.4809 - val_loss: 1.7043 - val_accuracy: 0.3375
Epoch 112/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.3515 - accuracy: 0.4847 - val_loss: 1.7897 - val_accuracy: 0.3258
Epoch 1

Epoch 163/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.2165 - accuracy: 0.5526 - val_loss: 1.6400 - val_accuracy: 0.3642
Epoch 164/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.2097 - accuracy: 0.5496 - val_loss: 1.7359 - val_accuracy: 0.3317
Epoch 165/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.2216 - accuracy: 0.5485 - val_loss: 1.7720 - val_accuracy: 0.3333
Epoch 166/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.2044 - accuracy: 0.5411 - val_loss: 1.7325 - val_accuracy: 0.3500
Epoch 167/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.2235 - accuracy: 0.5467 - val_loss: 1.7833 - val_accuracy: 0.3442
Epoch 168/5000
47/47 [==============================] - 0s 5ms/step - loss: 1.2163 - accuracy: 0.5359 - val_loss: 1.7176 - val_accuracy: 0.3558
Epoch 169/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.1940 - accuracy: 0.5608 - val_loss: 1.8298 - val_accuracy:

Epoch 220/5000
47/47 [==============================] - 0s 5ms/step - loss: 1.0743 - accuracy: 0.6102 - val_loss: 1.7475 - val_accuracy: 0.3800
Epoch 221/5000
47/47 [==============================] - 0s 5ms/step - loss: 1.0226 - accuracy: 0.6311 - val_loss: 1.8361 - val_accuracy: 0.3508
Epoch 222/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.0423 - accuracy: 0.6309 - val_loss: 1.7846 - val_accuracy: 0.3592
Epoch 223/5000
47/47 [==============================] - 0s 5ms/step - loss: 1.0238 - accuracy: 0.6272 - val_loss: 1.8452 - val_accuracy: 0.3467
Epoch 224/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.0634 - accuracy: 0.6056 - val_loss: 1.7113 - val_accuracy: 0.3783
Epoch 225/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.0078 - accuracy: 0.6394 - val_loss: 1.7356 - val_accuracy: 0.3617
Epoch 226/5000
47/47 [==============================] - 0s 6ms/step - loss: 1.0050 - accuracy: 0.6421 - val_loss: 1.8654 - val_accuracy:

...

47/47 [==============================] - 1s 11ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 5.2339 - val_accuracy: 0.3642
Epoch 2642/5000
47/47 [==============================] - 1s 11ms/step - loss: 0.0031 - accuracy: 0.9995 - val_loss: 5.2317 - val_accuracy: 0.3583
Epoch 2643/5000
47/47 [==============================] - 1s 11ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 5.2571 - val_accuracy: 0.3608
Epoch 2644/5000
47/47 [==============================] - 0s 10ms/step - loss: 0.0026 - accuracy: 0.9996 - val_loss: 5.2285 - val_accuracy: 0.3592
Epoch 2645/5000
47/47 [==============================] - 0s 7ms/step - loss: 0.0028 - accuracy: 0.9992 - val_loss: 5.2517 - val_accuracy: 0.3600
Epoch 2646/5000
47/47 [==============================] - 0s 5ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 5.3181 - val_accuracy: 0.3542
Epoch 2647/5000
47/47 [==============================] - 0s 5ms/step - loss: 0.0060 - accuracy: 0.9984 - val_loss: 5.2451 - val_accuracy: 0.36

47/47 [==============================] - 0s 10ms/step - loss: 0.0030 - accuracy: 0.9994 - val_loss: 5.2557 - val_accuracy: 0.3592
Epoch 2698/5000
47/47 [==============================] - 0s 6ms/step - loss: 0.0033 - accuracy: 0.9986 - val_loss: 5.3891 - val_accuracy: 0.3583
Epoch 2699/5000
47/47 [==============================] - 0s 10ms/step - loss: 0.0041 - accuracy: 0.9995 - val_loss: 5.2730 - val_accuracy: 0.3608
Epoch 2700/5000
47/47 [==============================] - 0s 10ms/step - loss: 0.0021 - accuracy: 0.9996 - val_loss: 5.2861 - val_accuracy: 0.3558
Epoch 2701/5000
47/47 [==============================] - 0s 6ms/step - loss: 0.0044 - accuracy: 0.9992 - val_loss: 5.2572 - val_accuracy: 0.3592
Epoch 2702/5000
47/47 [==============================] - 0s 7ms/step - loss: 0.0041 - accuracy: 0.9987 - val_loss: 5.2626 - val_accuracy: 0.3617
Epoch 2703/5000
47/47 [==============================] - 0s 9ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 5.2518 - val_accuracy: 0.355

47/47 [==============================] - 0s 10ms/step - loss: 0.0020 - accuracy: 0.9997 - val_loss: 5.2941 - val_accuracy: 0.3592
Epoch 2754/5000
47/47 [==============================] - 1s 11ms/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 5.2895 - val_accuracy: 0.3592
Epoch 2755/5000
47/47 [==============================] - 0s 8ms/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 5.2666 - val_accuracy: 0.3592
Epoch 2756/5000
47/47 [==============================] - 0s 5ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 5.3051 - val_accuracy: 0.3608
Epoch 2757/5000
47/47 [==============================] - 0s 5ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 5.2776 - val_accuracy: 0.3608
Epoch 2758/5000
47/47 [==============================] - 0s 6ms/step - loss: 0.0037 - accuracy: 0.9990 - val_loss: 5.2776 - val_accuracy: 0.3550
Epoch 2759/5000
47/47 [==============================] - 0s 5ms/step - loss: 0.0050 - accuracy: 0.9991 - val_loss: 5.2925 - val_accuracy: 0.3575

47/47 [==============================] - 0s 8ms/step - loss: 0.0021 - accuracy: 0.9996 - val_loss: 5.3096 - val_accuracy: 0.3658
Epoch 2810/5000
47/47 [==============================] - 0s 7ms/step - loss: 0.0031 - accuracy: 0.9993 - val_loss: 5.3008 - val_accuracy: 0.3583
Epoch 2811/5000
47/47 [==============================] - 0s 7ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 5.4403 - val_accuracy: 0.3583
Epoch 2812/5000
47/47 [==============================] - 0s 7ms/step - loss: 0.0055 - accuracy: 0.9990 - val_loss: 5.2937 - val_accuracy: 0.3600
Epoch 2813/5000
47/47 [==============================] - 0s 7ms/step - loss: 0.0019 - accuracy: 0.9997 - val_loss: 5.4130 - val_accuracy: 0.3633
Epoch 2814/5000
47/47 [==============================] - 0s 9ms/step - loss: 0.0048 - accuracy: 0.9991 - val_loss: 5.3126 - val_accuracy: 0.3625
Epoch 2815/5000
47/47 [==============================] - 0s 8ms/step - loss: 0.0027 - accuracy: 0.9993 - val_loss: 5.3861 - val_accuracy: 0.3567
E

47/47 [==============================] - 1s 11ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 5.3429 - val_accuracy: 0.3650
Epoch 2866/5000
47/47 [==============================] - 1s 13ms/step - loss: 0.0030 - accuracy: 0.9995 - val_loss: 5.3401 - val_accuracy: 0.3575
Epoch 2867/5000
47/47 [==============================] - 0s 10ms/step - loss: 0.0020 - accuracy: 0.9998 - val_loss: 5.3169 - val_accuracy: 0.3608
Epoch 2868/5000
47/47 [==============================] - 0s 10ms/step - loss: 0.0020 - accuracy: 0.9997 - val_loss: 5.3226 - val_accuracy: 0.3617
Epoch 2869/5000
47/47 [==============================] - 0s 6ms/step - loss: 0.0026 - accuracy: 0.9993 - val_loss: 5.3178 - val_accuracy: 0.3583
Epoch 2870/5000
47/47 [==============================] - 0s 6ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 5.3427 - val_accuracy: 0.3683
Epoch 2871/5000
47/47 [==============================] - 0s 6ms/step - loss: 0.0027 - accuracy: 0.9996 - val_loss: 5.3358 - val_accuracy: 0.36